# Description

Takes results of one example-based evaluation through Evaluator.exe and Preciser.exe and computes precision and recall.

### Imports

In [50]:
import pandas as pd
import numpy as np
from collections import defaultdict

# Settings

In [52]:
site = 'walmart' # 'webmd', 'walmart', 'guardian', 'cnn'
participants = ['P1', 'P2', 'P3', 'P4']
evaluation = 'gt-walmart'
dataset_evaluation_dir = 'C:/GazeMiningDataset/Dataset_evaluation/case-study'
dataset_stimuli_dir = r'C:/GazeMiningDataset/Dataset_stimuli'

# Build up pathes
labels_screencasts_filepath = dataset_evaluation_dir + '/' + evaluation + '-screencasts.csv'
labels_stimuli_filepath = dataset_evaluation_dir + '/' + evaluation + '-stimuli.csv'
contrib_filepath = dataset_evaluation_dir + '/' + evaluation + '-contrib.csv'
events_filepath = dataset_evaluation_dir + '/' + evaluation + '-events.csv'
stimuli_dir = dataset_stimuli_dir + '/' + site + '/stimuli'

# Load dataset

In [53]:
# Load evaluator labels of stimuli
stimuli_df = pd.read_csv(labels_stimuli_filepath)

# Load evaluator labels of screencasts
screencasts_df = pd.read_csv(labels_screencasts_filepath)

# Load contribution (this tells one which frames as contained in the stimuli contributed to the element in the task)
contrib_df = pd.read_csv(contrib_filepath)

# Load events
events_df = pd.read_csv(events_filepath, header=None, names=['timestamp', 'type', 'event'])

# Process events for general information

In [54]:
mode_changes_df = events_df[(events_df.type == 'mode_change')]

# Get timestamps
start_ts = int(mode_changes_df[(mode_changes_df.event == 'mode_start')].timestamp)
videos_ts = int(mode_changes_df[(mode_changes_df.event == 'mode_videos')].timestamp)
stimuli_ts = int(mode_changes_df[(mode_changes_df.event == 'mode_stimuli')].timestamp)
end_ts = int(mode_changes_df[(mode_changes_df.event == 'mode_end')].timestamp)

# Compute durations of modes
video_duration = 0
stimuli_duration = 0
if videos_ts > stimuli_ts:
    video_duration = end_ts - videos_ts
    stimuli_duration = videos_ts - stimuli_ts
else:
    video_duration = stimuli_ts - videos_ts
    stimuli_duration = end_ts - stimuli_ts
    
# Print results
print('Video Mode [s]: ' + str(video_duration/1000) + ', Stimuli Mode [s]: ' + str(stimuli_duration/1000))

Video Mode [s]: 356.671, Stimuli Mode [s]: 209.147


## Collect frames represented by marked stimuli

In [55]:
# Make groups by layer id from labeled stimuli
layers = stimuli_df.groupby(['layer_id'])

# Set of frames that are represented by the stimuli
marked_stimuli_frames = defaultdict(set) # participant_id -> set of frames

# Go over layer groups
for layer_id, data in layers:
    
    # Get ids of stimuli that are marked
    marked_stimuli_ids = data[data.label == 1]['stimulus_id']
    
    # Go over marked stimuli and collect all represented frames across shots per screencast
    for stimulus_id in marked_stimuli_ids:
        df = pd.read_csv(stimuli_dir + '/' + layer_id + '/' + str(stimulus_id) +'-shots.csv') # read in information about stimulus (which frames are contained...)
        for index, row in df.iterrows(): # go over contained shots and collect the frames
            frames = list(range(row['frame_idx_start'], row['frame_idx_end']+1, 1))
            participant_id = row['session_id'][:2].upper()
            marked_stimuli_frames[participant_id].update(frames) # put frames into the set, one set per screencast
            
# print(marked_stimuli_frames)

## Retrieve contribution

In [56]:
# There are some duplicated frames in the contrib files, fix that
# Note: This happens, when one frame is separated for stimuli discovery
# into more than one layer and the element is found on both layers in the evaluation
groups = contrib_df.groupby(['session'])
new_df = pd.DataFrame()
for key in groups.groups.keys():
    df = groups.get_group(key)
    df = df.drop_duplicates(subset='frame_idx')
    new_df = new_df.append(df, ignore_index=True)
contrib_df = new_df

print('Contrib Frame Count: ' + str(contrib_df.shape[0]))

Contrib Frame Count: 859


In [57]:
pos_contrib_count = contrib_df[contrib_df.label == 'POS_CONTRIB'].shape[0]
neg_contrib_count = contrib_df[contrib_df.label == 'NEG_CONTRIB'].shape[0]
neutral_count = contrib_df[contrib_df.label == 'NEUTRAL'].shape[0]

print('POS_CONTRIB: ' + str(pos_contrib_count))
print('NEG_CONTRIB: ' + str(neg_contrib_count))
print('NEUTRAL: ' + str(neutral_count))

POS_CONTRIB: 464
NEG_CONTRIB: 0
NEUTRAL: 395


## Collect frames marked in screencasts

In [58]:
# Marked frames of screencasts
marked_screencast_frames = {}
marked_screencast_frames['P1'] = screencasts_df.index[screencasts_df.P1 == 1]
marked_screencast_frames['P2'] = screencasts_df.index[screencasts_df.P2 == 1]
marked_screencast_frames['P3'] = screencasts_df.index[screencasts_df.P3 == 1]
marked_screencast_frames['P4'] = screencasts_df.index[screencasts_df.P4 == 1]

print(marked_screencast_frames)

# Count frames
p1_no = len(screencasts_df.index[screencasts_df.P1 == 0])
p1_yes = len(screencasts_df.index[screencasts_df.P1 == 1])
p2_no = len(screencasts_df.index[screencasts_df.P2 == 0])
p2_yes = len(screencasts_df.index[screencasts_df.P2 == 1])
p3_no = len(screencasts_df.index[screencasts_df.P3 == 0])
p3_yes = len(screencasts_df.index[screencasts_df.P3 == 1])
p4_no = len(screencasts_df.index[screencasts_df.P4 == 0])
p4_yes = len(screencasts_df.index[screencasts_df.P4 == 1])

print('Annotated Frames: ' + str(p1_no + p1_yes + p2_no + p2_yes + p3_no + p3_yes + p4_no + p4_yes))
print('Display AOI: ' + str(p1_yes + p2_yes + p3_yes + p4_yes))
print('Do not display AOI: ' + str(p1_no + p2_no + p3_no + p4_no))

{'P1': Int64Index([132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,
            145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 188,
            189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201,
            202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 217, 218],
           dtype='int64'), 'P2': Int64Index([ 844,  845,  846,  847,  848,  849,  850,  851,  852,  853,
            ...
            1027, 1028, 1029, 1030, 1031, 1032, 1033, 1039, 1040, 1041],
           dtype='int64', length=191), 'P3': Int64Index([194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206,
            207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,
            362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374,
            375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387,
            388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400,
            401, 404, 405, 406, 629, 630, 851, 852, 853, 854

# Calculate precision and recall

In [11]:
# Compare both dictionaries
stimuli_frames_count = 0;
recall_count = 0
screencast_frames_count = 0
for participant in participants:
    
    # Get frames marked (indirectly) in stimuli and screencasts
    stimuli_frames = set(marked_stimuli_frames[participant])
    screencast_frames = set(marked_screencast_frames[participant])
    stimuli_frames_count += len(stimuli_frames)
    screencast_frames_count += len(screencast_frames)
    
    # Compute recall
    recall_count += len(screencast_frames.intersection(stimuli_frames)) # frames that are contained in both sets

print('Stimuli Frames Count: ' + str(stimuli_frames_count))
print('Screencast Frames Count: ' + str(screencast_frames_count))
print('Precision: ' + str(pos_contrib_count / (pos_contrib_count + neg_contrib_count)))
print('Recall: ' + str(recall_count / screencast_frames_count))

Stimuli Frames Count: 708
Screencast Frames Count: 316
Precision: 0.9269662921348315
Recall: 1.0
